特征选择主要有两个功能

1. 减少特征数量、降维，使模型泛化能力更强，减少过拟合
2. 增强对特征和特征值之间的理解

# 去掉取值变化小的特征

特征是否发散：如果一个特征不发散，例如方差接近于0，也就是说样本在这个特征上基本上没有差异，这个特征对于样本的区分并没有什么用。

假设某特征的特征值只有0和1，并且在所有输入样本中，95%的实例的该特征取值都是1，那就可以认为这个特征作用不大。如果100%都是1，那这个特征就没意义了。可以把它作为特征选择的预处理。

In [61]:
from sklearn.feature_selection import VarianceThreshold
X = [[0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 1, 1], [0, 1, 0], [0, 1, 1]]
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
X_new = sel.fit_transform(X)
print(sel.variances_ )
print(X_new)

[ 0.13888889  0.22222222  0.25      ]
[[0 1]
 [1 0]
 [0 0]
 [1 1]
 [1 0]
 [1 1]]


VarianceThreshold 移除了第一列特征，第一列中特征值为0的概率达到了5/6.

# 单变量特征选择

单变量特征选择能够对每一个特征进行测试，衡量该特征和响应变量之间的关系，根据得分扔掉不好的特征。

- 对于分类问题（y离散）,可采用：卡方检验， f_classif, mutual_info_classif, 互信息
- 对于回归问题（y连续），可采用：皮尔森相关系数，f_regression, mutual_info_regression, 最大信息系数

## Pearson相关系数

$$r = \frac{1}{n-1} \sum _{i=1}^n (\frac{X_i - \overline{X}} {s_X})(\frac{Y_i - \overline{Y}}{s_Y})$$

皮尔森相关系数是一种最简单的，能帮助理解特征和响应变量之间关系的方法，该方法衡量的是变量之间的线性相关性，结果的取值区间为[-1，1]

In [1]:
import numpy as np
from scipy.stats import pearsonr

np.random.seed(0)
size = 300
x = np.random.normal(0, 1, size)
# pearson(x, y)的输入为特征矩阵和目标向量
print("Lower noise", pearsonr(x, x + np.random.normal(0, 1, size)))
print("Higher noise", pearsonr(x, x + np.random.normal(1, 10, size)))
# 输出为元组(score, p-value)的数组

Lower noise (0.71824836862138408, 7.3240173129983507e-49)
Higher noise (0.057964292079338155, 0.31700993885324752)


Pearson相关系数的一个明显缺陷：只对线性关系敏感

In [2]:
x = np.random.uniform(-1, 1, 10000)
print(pearsonr(x, x**2)[0])

-0.0209394101943


## 互信息和最大信息系数

$I(X, Y) = \sum _{y\in{Y}}\sum _{x\in{X}}p(x, y)log(\frac{p(x,y)}{p(x)(y)})$

经典互信息的两大不足：
1. 它不属于度量方式，也没有办法归一化，在不同数据集上的结果无法做比较；
2. 对于连续变量的计算不是很方便（X和Y都是集合，x，y都是离散的取值），通常变量需要先离散化，而互信息的结果对离散化的方式很敏感。

最大信息系数克服了这两个问题。它首先寻找一种最优的离散化方式，然后把互信息取值转换成一种度量方式，取值区间在[0，1]。

In [50]:
from minepy import MINE

m = MINE()
x = np.random.uniform(-1, 1, 10000)
m.compute_score(x, x**2)
print(round(m.mic(), 4))

1.0


## 卡方(Chi2)检验

In [56]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

iris = load_iris()
X, y = iris.data, iris.target
print(X.shape)
X_new = SelectKBest(chi2, k=2).fit_transform(X, y)
print(X_new.shape)

(150, 4)
(150, 2)


## 基于学习模型的特征排序

基于树的方法比较易于使用，因为他们对非线性关系的建模比较好，并且不需要太多的调试。但要注意过拟合问题，因此树的深度最好不要太大，再就是运用交叉验证。

In [3]:
from sklearn.cross_validation import cross_val_score, ShuffleSplit
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor

boston = load_boston()
X = boston["data"]
Y = boston["target"]
names = boston["feature_names"]

rf = RandomForestRegressor(n_estimators=20, max_depth=4)
scores = []
for i in range(X.shape[1]):
    score = cross_val_score(rf, X[:, i:i+1], Y, scoring="r2", cv=ShuffleSplit(len(X), 3, 0.3))
    scores.append((round(np.mean(score), 3), names[i]))
print(sorted(scores, reverse=True))

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


[(0.66700000000000004, 'LSTAT'), (0.57599999999999996, 'RM'), (0.42199999999999999, 'NOX'), (0.40899999999999997, 'INDUS'), (0.38900000000000001, 'PTRATIO'), (0.33100000000000002, 'TAX'), (0.20899999999999999, 'RAD'), (0.20000000000000001, 'ZN'), (0.155, 'CRIM'), (0.14199999999999999, 'B'), (0.111, 'DIS'), (0.10000000000000001, 'AGE'), (0.017000000000000001, 'CHAS')]


# 线性模型和正则化

，这种方法好像在一些地方叫做wrapper类型，大概意思是说，特征排序模型和机器学习模型是耦盒在一起的，对应的非wrapper类型的特征选择方法叫做filter类型。

在噪音不多的数据上，或者是数据量远远大于特征数的数据上，如果特征之间相对来说是比较独立的，那么即便是运用最简单的线性回归模型也一样能取得非常好的效果。

In [4]:
from sklearn.linear_model import LinearRegression

size = 5000

X = np.random.normal(0, 1, (size, 3))
Y = X[:, 0] + 2*X[:, 1] + np.random.normal(0, 2, size)
lr = LinearRegression()
lr.fit(X, Y)

# A helper method for pretty-printing linear models
def pretty_print_linear(coefs, names=None, sort=False):
    if names == None:
        names = ["X%s" % x for x in range(len(coefs))]
    lst = zip(coefs, names)
    if sort:
        lst = sorted(lst, key=lambda x:-np.abs(X[0]))
    return " + ".join("%s * %s" % (round(coef, 3), name) for coef, name in lst)

print("Linear model:", pretty_print_linear(lr.coef_))

Linear model: 0.993 * X0 + 2.052 * X1 + 0.052 * X2


在很多实际的数据当中，往往存在多个互相关联的特征，这时候模型就会变得不稳定，数据中细微的变化就可能导致模型的巨大变化，这会让模型的预测变得困难，这种现象也称为多重共线性。
例如，假设我们有个数据集，它的真实模型应该是Y=X1+X2，当我们观察的时候，发现Y’=X1+X2+e，e是噪音。如果X1和X2之间存在线性关系，例如X1约等于X2，这个时候由于噪音e的存在，我们学到的模型可能就不是Y=X1+X2了，有可能是Y=2X1，或者Y=-X1+3X2。

In [8]:
size = 100

X_seed = np.random.normal(0, 1, size)
x1 = X_seed + np.random.normal(0, .1, size)
x2 = X_seed + np.random.normal(0, .1, size)
x3 = X_seed + np.random.normal(0, .1, size)

Y = x1 + x2 + x3 + np.random.normal(0, 1, size)
X = np.array([x1, x2, x3]).T

lr = LinearRegression()
lr.fit(X, Y)
print("Linear model:", pretty_print_linear(lr.coef_))

Linear model: 1.188 * X0 + 1.28 * X1 + 0.577 * X2


系数之和接近3，基本上和上上个例子的结果一致，实际上所有特征与输出变量之间的影响是均等的。

## 正则化模型

正则化就是把额外的约束或者惩罚项加到已有模型（损失函数）上，以防止过拟合并提高泛化能力。损失函数由原来的E(X,Y)变为E(X,Y)+alpha||w||，w是模型系数组成的向量（有些地方也叫参数parameter，coefficients），||·||一般是L1或者L2范数，alpha是一个可调的参数，控制着正则化的强度。当用在线性模型上时，L1正则化和L2正则化也称为Lasso和Ridge。

## L1正则化/Lasso

L1正则化将系数w的l1范数作为惩罚项加到损失函数上，由于正则项非零，这就迫使那些弱的特征所对应的系数变成0。因此L1正则化往往会使学到的模型很稀疏（系数w经常为0），这个特性使得L1正则化成为一种很好的特征选择方法。

在波士顿房价数据上运行了Lasso，其中参数alpha是通过grid search进行优化的

In [17]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel

iris = load_iris()
X, y = iris.data, iris.target
print(X.shape)

lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, y)
model = SelectFromModel(lsvc, prefit=True)
X_new = model.transform(X)
print(X_new.shape)

(150, 4)
(150, 3)


使用featur_selection库的SelectFromModel类结合

对于SVM和逻辑回归，参数C控制稀疏性：C越小，被选中的特征越少。对于Lasso，参数alpha越大，被选中的特征越少。

基于L1的稀疏模型的局限在于，当面对一组互相关的特征时，它们只会选择其中一项特征。为了减轻该问题的影响可以使用随机化技术，通过_多次重新估计稀疏模型来扰乱设计矩阵_，或通过_多次下采样数据来统计一个给定的回归量被选中的次数_

## L2正则化/Ridge regression

L2正则化将系数向量的L2范数添加到了损失函数中。由于L2惩罚项中系数是二次方的，这使得L2和L1有着诸多差异，最明显的一点就是，L2正则化会让系数的取值变得平均。

L2正则化对于特征选择来说一种稳定的模型，不像L1正则化那样，系数会因为细微的数据变化而波动。

In [18]:
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
size = 100

#We run the method 10 times with different random seeds
for i in range(10):
    print("Random seed %s" % i)
    np.random.seed(seed=i)
    X_seed = np.random.normal(0, 1, size)
    X1 = X_seed + np.random.normal(0, .1, size)
    X2 = X_seed + np.random.normal(0, .1, size)
    X3 = X_seed + np.random.normal(0, .1, size)
    Y = X1 + X2 + X3 + np.random.normal(0, 1, size)
    X = np.array([X1, X2, X3]).T

    lr = LinearRegression()
    lr.fit(X,Y)
    print("Linear model:", pretty_print_linear(lr.coef_))

    ridge = Ridge(alpha=10)
    ridge.fit(X,Y)
    print("Ridge model:", pretty_print_linear(ridge.coef_))
    print()

Random seed 0
Linear model: 0.728 * X0 + 2.309 * X1 + -0.082 * X2
Ridge model: 0.938 * X0 + 1.059 * X1 + 0.877 * X2

Random seed 1
Linear model: 1.152 * X0 + 2.366 * X1 + -0.599 * X2
Ridge model: 0.984 * X0 + 1.068 * X1 + 0.759 * X2

Random seed 2
Linear model: 0.697 * X0 + 0.322 * X1 + 2.086 * X2
Ridge model: 0.972 * X0 + 0.943 * X1 + 1.085 * X2

Random seed 3
Linear model: 0.287 * X0 + 1.254 * X1 + 1.491 * X2
Ridge model: 0.919 * X0 + 1.005 * X1 + 1.033 * X2

Random seed 4
Linear model: 0.187 * X0 + 0.772 * X1 + 2.189 * X2
Ridge model: 0.964 * X0 + 0.982 * X1 + 1.098 * X2

Random seed 5
Linear model: -1.291 * X0 + 1.591 * X1 + 2.747 * X2
Ridge model: 0.758 * X0 + 1.011 * X1 + 1.139 * X2

Random seed 6
Linear model: 1.199 * X0 + -0.031 * X1 + 1.915 * X2
Ridge model: 1.016 * X0 + 0.89 * X1 + 1.091 * X2

Random seed 7
Linear model: 1.474 * X0 + 1.762 * X1 + -0.151 * X2
Ridge model: 1.018 * X0 + 1.039 * X1 + 0.901 * X2

Random seed 8
Linear model: 0.084 * X0 + 1.88 * X1 + 1.107 * X2
Ridg

# 随机森林

随机森林具有准确率高、鲁棒性好、易于使用等优点，这使得它成为了目前最流行的机器学习算法之一。随机森林提供了两种特征选择的方法：mean decrease impurity和mean decrease accuracy。

## 平均不纯度减少 mean decrease impurity

利用不纯度可以确定节点（最优条件），对于分类问题，通常采用基尼不纯度或者信息增益，对于回归问题，通常采用的是方差或者最小二乘拟合。当训练决策树的时候，可以计算出每个特征减少了多少树的不纯度。对于一个决策树森林来说，可以算出每个特征平均减少了多少不纯度，并把它平均减少的不纯度作为特征选择的值。

In [24]:
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor
import numpy as np

boston = load_boston()
X = boston["data"]
Y = boston["target"]
names = boston["feature_names"]
rf = RandomForestRegressor()
rf.fit(X, Y)
print("Features sorted by their score:")
print(sorted(zip(map(lambda x : round(x, 4), rf.feature_importances_), names), reverse=True))

Features sorted by their score:
[(0.51880000000000004, 'LSTAT'), (0.2858, 'RM'), (0.061699999999999998, 'DIS'), (0.042999999999999997, 'NOX'), (0.0258, 'CRIM'), (0.015699999999999999, 'AGE'), (0.0138, 'PTRATIO'), (0.0109, 'INDUS'), (0.0106, 'TAX'), (0.0097999999999999997, 'B'), (0.0028999999999999998, 'RAD'), (0.00089999999999999998, 'ZN'), (0.00040000000000000002, 'CHAS')]


不纯度的方法的特点：
1. 这种方法存在偏向，对具有更多类别的变量会更有利；
2. 对于存在关联的多个特征，其中任意一个都可以作为指示器（优秀的特征），并且一旦某个特征被选择之后，其他特征的重要度就会急剧下降，因为不纯度已经被选中的那个特征降下来了，其他的特征就很难再降低那么多不纯度了，这样一来，只有先被选中的那个特征重要度很高，其他的关联特征重要度往往较低。

特征随机选择方法稍微缓解了这个问题，但总的来说并没有完全解决。

In [29]:
size = 10000
np.random.seed(seed=10)
X_seed = np.random.normal(0, 1, size)
x0 = X_seed + np.random.normal(0, .1, size)
x1 = X_seed + np.random.normal(0, .1, size)
x2 = X_seed + np.random.normal(0, .1, size)
X = np.array([x0, x1, x2]).T
y = x0 + x1 + x2

rf = RandomForestRegressor(n_estimators=20, max_features=2)
rf.fit(X, y)
print("Score for x0, x1, x2:", list(map(lambda x: round(x, 3), rf.feature_importances_)))

Score for x0, x1, x2: [0.27200000000000002, 0.54800000000000004, 0.17899999999999999]


Ps. 关联特征的打分存在不稳定的现象，这不仅仅是随机森林特有的，大多数基于模型的特征选择方法都存在这个问题。

## 平均精度率减少 Mean decrease accuracy

直接度量每个特征对模型精确率的影响。主要思路是打乱每个特征的特征值顺序，并且度量顺序变动对模型的精确率的影响。很明显，对于不重要的变量来说，打乱顺序对模型的精确率影响不会太大，但是对于重要的变量来说，打乱顺序就会降低模型的精确率。

In [39]:
from sklearn.cross_validation import ShuffleSplit
from sklearn.metrics import r2_score
from collections import defaultdict

X = boston["data"]
Y = boston["target"]

rf = RandomForestRegressor()
scores = defaultdict(list)

for train_idx, test_idx in ShuffleSplit(len(X), 100, .3):
    X_train, X_test = X[train_idx], X[test_idx]
    Y_train, Y_test = Y[train_idx], Y[test_idx]
    r = rf.fit(X_train, Y_train)
    acc = r2_score(Y_test, rf.predict(X_test))
    for i in range(X.shape[1]):
        X_t = X_test.copy()
        np.random.shuffle(X_t[:, i])
        shuffle_acc = r2_score(Y_test, rf.predict(X_t))
        scores[names[i]].append((acc-shuffle_acc)/acc)
print("Features sorted by their score:")
print(sorted([(round(np.mean(score), 4), feat) for feat, score in scores.items()], reverse=True))

Features sorted by their score:
[(0.73480000000000001, 'LSTAT'), (0.57120000000000004, 'RM'), (0.084000000000000005, 'DIS'), (0.041799999999999997, 'CRIM'), (0.039100000000000003, 'NOX'), (0.019900000000000001, 'PTRATIO'), (0.016199999999999999, 'TAX'), (0.0117, 'AGE'), (0.0047999999999999996, 'B'), (0.0044999999999999997, 'INDUS'), (0.0033999999999999998, 'RAD'), (0.00059999999999999995, 'CHAS'), (0.00029999999999999997, 'ZN')]


# 两种顶层特征选择算法

之所以叫做顶层，是因为他们都是建立在基于模型的特征选择方法基础之上的，例如回归和SVM，在不同的子集上建立模型，然后汇总最终确定特征得分。

## 稳定性选择 Staility selection

稳定性选择是一种基于二次抽样和选择算法相结合较新的方法，选择算法可以是回归、SVM或其他类似的方法。它的主要思想是在不同的数据子集和特征子集上运行特征选择算法，不断的重复，最终汇总特征选择结果，比如可以统计某个特征被认为是重要特征的频率（被选为重要特征的次数除以它所在的子集被测试的次数）。理想情况下，重要特征的得分会接近100%。稍微弱一点的特征得分会是非0的数，而最无用的特征得分将会接近于0。

sklearn在随机lasso和随机逻辑回归中有对稳定性选择的实现。

In [47]:
from sklearn.linear_model import RandomizedLasso
from sklearn.datasets import load_boston
boston = load_boston()

# Data gets scaled automatically by sklearn's implementation
X = boston["data"]
Y = boston["target"]
names = boston["feature_names"]

rlasso = RandomizedLasso(alpha=0.025)
rlasso.fit(X, Y)

print("Features sorted by their score:")
print(sorted(zip(map(lambda x: round(x, 4), rlasso.scores_), names), reverse=True))

C:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class RandomizedLasso is deprecated; The class RandomizedLasso is deprecated in 0.19 and will be removed in 0.21.
  warnings.warn(msg, category=DeprecationWarning)


Features sorted by their score:
[(1.0, 'RM'), (1.0, 'PTRATIO'), (1.0, 'LSTAT'), (0.64000000000000001, 'CHAS'), (0.64000000000000001, 'B'), (0.40500000000000003, 'TAX'), (0.39500000000000002, 'CRIM'), (0.23000000000000001, 'NOX'), (0.22, 'DIS'), (0.11, 'INDUS'), (0.065000000000000002, 'ZN'), (0.014999999999999999, 'AGE'), (0.01, 'RAD')]


稳定性选择对于克服过拟合和对数据理解来说都是有帮助的：总的来说，好的特征不会因为有相似的特征、关联特征而得分为0，这跟Lasso是不同的。对于特征选择任务，在许多数据集和环境下，稳定性选择往往是性能最好的方法之一。

## 递归特征消除 Recursive feature elimination(RFE)

递归特征消除的主要思想是反复的构建模型（如SVM或者回归模型）然后选出最好的（或者最差的）的特征（可以根据系数来选），把选出来的特征放到一遍，然后在剩余的特征上重复这个过程，直到所有特征都遍历了。这个过程中特征被消除的次序就是特征的排序。

RFE的稳定性很大程度上取决于在迭代的时候底层用哪种模型。例如，假如RFE采用的普通的回归，没有经过正则化的回归是不稳定的，那么RFE就是不稳定的；假如采用的是Ridge，而用Ridge正则化的回归是稳定的，那么RFE就是稳定的。

Sklearn提供了RFE包，可以用于特征消除，还提供了RFECV，可以通过交叉验证来对的特征进行排序。

In [48]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
rfe = RFE(lr, n_features_to_select=1)
rfe.fit(X, Y)

print("Feature sorted by their ranks:")
print(sorted(zip(map(lambda x: round(x, 4), rfe.ranking_), names)))

Feature sorted by their ranks:
[(1, 'NOX'), (2, 'RM'), (3, 'CHAS'), (4, 'PTRATIO'), (5, 'DIS'), (6, 'LSTAT'), (7, 'RAD'), (8, 'CRIM'), (9, 'INDUS'), (10, 'ZN'), (11, 'TAX'), (12, 'B'), (13, 'AGE')]


# 一个完整的例子

数据产生公式:

$y = 10sin(πx_1x_2) + 20(x_3-0.5)^2 + 10x_4 + 5x_5 + e$

X1到X5是由单变量分布生成的，e是标准正态变量N(0,1)。另外，原始的数据集中含有5个噪音变量 X5,…,X10，跟响应变量是独立的。我们增加了4个额外的变量X11,…X14，分别是X1,…,X4的关联变量，通过f(x)=x+N(0,0.01)生成，这将产生大于0.999的关联系数。这样生成的数据能够体现出不同的特征排序方法应对关联特征时的表现。

In [25]:
from sklearn.datasets import load_boston
from sklearn.linear_model import (LinearRegression, Ridge, Lasso, RandomizedLasso)
from sklearn.feature_selection import RFE, f_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from minepy import MINE

np.random.seed(0)

size = 750
X = np.random.uniform(0, 1, (size, 14))
Y = 10 * np.sin(np.pi * X[:, 0] * X[:, 1]) + 20 * (X[:, 2] - .5) ** 2 + 10 * X[:, 3] + 5 * X[:, 4] + np.random.normal(0, 1)

# Add 3 additional correlated variables (correlated with X1-X3)
X[:, 10:] = X[:, :4] + np.random.normal(0, .25, (size, 4))
names = ["x%s" % i for i in range(1, 15)]

ranks = {}

def rank_to_dict(ranks, names, order=1):
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order * np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x, 2), ranks)
    return dict(zip(names, ranks))

lr = LinearRegression(normalize=True)
lr.fit(X, Y)
ranks["reg"] = rank_to_dict(np.abs(lr.coef_), names)

ridge = Ridge(alpha=7)
ridge.fit(X, Y)
ranks["Ridge"] = rank_to_dict(np.abs(ridge.coef_), names)

lasso = Lasso(alpha=.05)
lasso.fit(X, Y)
ranks["Lasso"] = rank_to_dict(np.abs(lasso.coef_), names)

rlasso = RandomizedLasso(alpha=0.04)
rlasso.fit(X, Y)
ranks["Stab"] = rank_to_dict(np.abs(rlasso.scores_), names)

# stop search when 5 features are left (they will get equal scores)
rfe = RFE(lr, n_features_to_select=5)
rfe.fit(X, Y)
ranks["RFE"] = rank_to_dict(list(map(float, rfe.ranking_)), names, order=-1)

rf = RandomForestRegressor()
rf.fit(X, Y)
ranks["RF"] = rank_to_dict(rf.feature_importances_, names)

f, pval = f_regression(X, Y, center=True)
ranks["Corr."] = rank_to_dict(f, names)

mine = MINE()
mic_scores = []
for i in range(X.shape[1]):
    mine.compute_score(X[:, i], Y)
    m = mine.mic()
    mic_scores.append(m)
    
ranks["MIC"] = rank_to_dict(mic_scores, names)

r = {}
for name in names:
    r[name] = round(np.mean([ranks[method][name] for method in ranks.keys()]), 2) 
    
methods = sorted(ranks.keys())
ranks["Mean"] = r
methods.append("Mean")

print("\t%s" % "\t".join(methods))
for name in names:
    print("%s\t%s" % (name, "\t".join(map(str, [ranks[method][name] for method in methods]))))

C:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class RandomizedLasso is deprecated; The class RandomizedLasso is deprecated in 0.19 and will be removed in 0.21.
  warnings.warn(msg, category=DeprecationWarning)


	Corr.	Lasso	MIC	RF	RFE	Ridge	Stab	reg	Mean
x1	0.3	0.69	0.39	0.6	1.0	0.79	0.74	0.81	0.66
x2	0.44	0.73	0.61	0.69	1.0	0.75	0.72	0.76	0.71
x3	0.0	0.0	0.35	0.11	0.89	0.08	0.0	0.11	0.19
x4	1.0	1.0	1.0	1.0	1.0	1.0	1.0	1.0	1.0
x5	0.1	0.45	0.21	0.19	1.0	0.52	0.46	0.48	0.43
x6	0.0	0.0	0.01	0.0	0.56	0.03	0.0	0.03	0.08
x7	0.01	0.0	0.08	0.01	0.11	0.01	0.0	0.0	0.03
x8	0.02	0.0	0.06	0.01	0.67	0.07	0.0	0.05	0.11
x9	0.01	0.0	0.1	0.0	0.22	0.01	0.0	0.0	0.04
x10	0.0	0.0	0.05	0.0	0.33	0.01	0.0	0.02	0.05
x11	0.12	0.0	0.18	0.03	1.0	0.0	0.31	0.1	0.22
x12	0.23	0.0	0.23	0.0	0.44	0.08	0.33	0.02	0.17
x13	0.0	0.0	0.0	0.01	0.78	0.06	0.0	0.08	0.12
x14	0.47	0.02	0.47	0.01	0.0	0.13	0.4	0.0	0.19


- Lasso能够挑出一些优质特征，同时让其他特征的系数趋于0。当如需要减少特征数的时候它很有用，但是对于数据理解来说不是很好用。（例如在结果表中，X11,X12,X13的得分都是0，好像他们跟输出变量之间没有很强的联系，但实际上不是这样的）
- MIC对特征一视同仁，这一点上和关联系数有点像，另外，它能够找出X3和响应变量之间的非线性关系。
- 随机森林基于不纯度的排序结果非常鲜明，在得分最高的几个特征之后的特征，得分急剧的下降。从表中可以看到，得分第三的特征比第一的小4倍。而其他的特征选择算法就没有下降的这么剧烈。
- Ridge将回归系数均匀的分摊到各个关联变量上，从表中可以看出，X11,…,X14和X1,…,X4的得分非常接近。
- 稳定性选择常常是一种既能够有助于理解数据又能够挑出优质特征的这种选择，在结果表中就能很好的看出。像Lasso一样，它能找到那些性能比较好的特征（X1，X2，X4，X5），同时，与这些特征关联度很强的变量也得到了较高的得分。

# 总结

- Filter：过滤法，按照发散性或者相关性对各个特征进行评分，设定阈值或者待选择阈值的个数，选择特征。
- Wrapper：包装法，根据目标函数（通常是预测效果评分），每次选择若干特征，或者排除若干特征。
- Embedded：嵌入法，先使用某些机器学习的算法和模型进行训练，得到各个特征的权值系数，根据系数从大到小选择特征。类似于Filter方法，但是是通过训练来确定特征的优劣。

# Tips

# 参考

- https://www.cnblogs.com/-Sai-/p/6860702.html
- https://www.cnblogs.com/stevenlk/p/6543628.html